In [1]:
# Import findspark
import findspark
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')
findspark.find()

'/home/ubuntu/spark-3.3.1-bin-hadoop3'

In [2]:
from pyspark.sql import SparkSession

# The entry point into all functionality in Spark is the SparkSession class.
spark = (SparkSession
	.builder
	.appName("DS5110/CS5501: my awesome Spark program")
	.master("spark://172.31.88.97:7077")
	.config("spark.executor.memory", "1024M")
	.getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/08 16:48:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
# Read data
full_data = (spark.read
        .format('csv')
        .option('inferSchema', True)
        .option('header', True)
        .load('hdfs://172.31.88.97:9000/cleaned_data_new.csv')
     )

In [5]:
full_data.head()

Row(legId='222cfd6d1b0d5732602a3e82ad7730c3', searchDate=0, flightDate=45, destinationAirport='BOS', baseFare=65.48, seatsRemaining=4, totalTravelDistance=947, durationSeconds=17400, dateDelta=45)

In [13]:
full_data.count()

39435236

In [31]:
subset_data  = full_data.sample(fraction=0.1)

In [32]:
subset_data.count()

3942563

## Train test split
# NEEDS TO BE STANDARDIZED ACROSS IMPLEMENTATIONS

In [61]:
(training_data, test_data) = subset_data.randomSplit([0.7, 0.3])

## Random Forest

In [62]:
from pyspark.ml.feature import VectorAssembler

numericCols = ['dateDelta', 'seatsRemaining', 'totalTravelDistance', 'durationSeconds']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
train = assembler.transform(training_data)
test = assembler.transform(test_data)

24/04/08 17:42:46 ERROR OWLQN: Failure! Resetting history: breeze.optimize.NaNHistory: 


24/04/08 17:46:39 ERROR OWLQN: Failure! Resetting history: breeze.optimize.NaNHistory: 


24/04/08 17:47:21 WARN TaskSetManager: Lost task 9.0 in stage 2295.0 (TID 27781) (172.31.94.13 executor 2): java.net.ConnectException: Call From ip-172-31-94-13/172.31.94.13 to ip-172-31-88-97.ec2.internal:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:828)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1616)
	at org.apache.hadoop.ipc.Client.call(Client.java:1558)
	at org.apache.hadoop.ipc.Client.ca

In [35]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(featuresCol = 'features', labelCol='baseFare')
model = rf.fit(train)

In [42]:
predictions = model.transform(test)

In [44]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol='baseFare', predictionCol='prediction', metricName='rmse'
)

In [47]:
rmse = evaluator.evaluate(predictions)

In [48]:
rmse

148.39153072138427

## K-Means

In [58]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator, MulticlassClassificationEvaluator

In [ ]:
# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(train)

# Make predictions
predictions = model.transform(test)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.7215018401621854
Cluster Centers: 
[2.71430356e+01 6.40547939e+00 1.15242095e+03 1.56616018e+04]
[2.73867929e+01 6.00535333e+00 2.16721632e+03 3.69557006e+04]


## SVM

In [53]:
from pyspark.ml.feature import StringIndexer

label_stringIdx = StringIndexer(inputCol = 'baseFare', outputCol = 'labelIndex')
training_data = label_stringIdx.fit(training_data).transform(training_data)
test_data = label_stringIdx.fit(test_data).transform(test_data)

In [54]:
numericCols = ['dateDelta', 'seatsRemaining', 'totalTravelDistance', 'durationSeconds', 'baseFare']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
train = assembler.transform(training_data)
test = assembler.transform(test_data)

In [60]:
from pyspark.ml.classification import LinearSVC, OneVsRest

# initiate base classifier
lsvc = LinearSVC(maxIter=10, regParam=0.1)

# initiate one vs. rest classifier
ovr = OneVsRest(classifier=lsvc, featuresCol = 'features', labelCol='labelIndex')

# Fit model
model = ovr.fit(train)

# predict
predictions = model.transform(test)

# initialize evaluator
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

# compute classification error
accuracy = evlauator.evaluate(predictions)
print(f'Test error = {accuracy}')

24/04/08 17:31:59 ERROR OWLQN: Failure! Resetting history: breeze.optimize.NaNHistory: 


24/04/08 17:32:59 ERROR OWLQN: Failure! Resetting history: breeze.optimize.NaNHistory: 


24/04/08 17:38:44 ERROR OWLQN: Failure! Resetting history: breeze.optimize.NaNHistory: 
24/04/08 17:38:50 ERROR OWLQN: Failure! Resetting history: breeze.optimize.NaNHistory: 


KeyboardInterrupt: 

## KNN

In [ ]:
# Imports
from pyspark import SparkConf, SparkContext
from sklearn.neighbors import NearestNeighbors
 
# Input a Spark object containing
# all vectors, called myvecs
myvecs.cache()
 
# Create kNN tree locally, and broadcast
myvecscollected = myvecs.collect()
knnobj = NearestNeighbors().fit(myvecscollected)
bc_knnobj = sc.broadcast(knnobj)
 
# Get neighbors for each point, distributedly
results = myvecs.map(lambda x: bc_knnobj.value.kneighbors(x))